In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

from poke_env.teambuilder.teambuilder import Teambuilder
from poke_env.teambuilder.teambuilder_pokemon import TeambuilderPokemon

from tqdm import tqdm

import pandas as pd
import json
import pandas as pd
import numpy as np

from typing import List, Tuple, Optional

from copy import deepcopy

In [2]:
class PokeCache:
    """

    """

    def __init__(self, fpath: Optional[str] = None):
        self.dex_cache = {}
        self.species_cache = {}

        if fpath is not None:
            self.load_from_file(fpath)

    def load_from_file(self, fpath: str):
        """

        """
        with open(fpath, "r") as f:
            contents = f.read()
            cache = json.loads(contents)

        self.dex_cache = cache["pokedex"]
        self.species_cache = cache["species"]

    def save_to_file(self, fpath: str):
        """

        """

        with open(fpath, "w") as f:
            output = {}
            output["pokedex"] = self.dex_cache
            output["species"] = self.species_cache

            contents = json.dumps(output)
            f.write(contents)

In [3]:
def build_random_team(
    possible_pokemon: List[str],
    level_options: List[int] = [80], #[5, 14, 20, 23, 27, 31, 35, 39, 43, 50],
    random_seed: Optional[int] = None,
    cache: Optional[PokeCache] = None
) -> str:
    """

    """

    if cache is None:
        cache = PokeCache()

    rng = np.random.default_rng(seed=random_seed)

    team_size = min(rng.integers(low=1, high=12, size=1), 6)
    poke_selection = rng.choice(possible_pokemon, team_size, replace=False)
    level = rng.choice(level_options)

    team = []
    for poke in poke_selection:
        if poke.strip().lower() not in cache.dex_cache.keys():
            pokedex_url = f"https://pokeapi.co/api/v2/pokemon/{poke.strip().lower()}/"
            pokedex_entry_response = requests.get(pokedex_url)

            if pokedex_entry_response.status_code != 200:
                print(pokedex_entry_response.status_code)
                print(pokedex_entry_response.content)
                raise RuntimeError(f"Error when trying to pull pokedex data for: {poke}")

            pokedex_entry = pokedex_entry_response.json()

            cache.dex_cache[poke.strip().lower()] = pokedex_entry

        generic_name = cache.dex_cache[poke.strip().lower()]["species"]["name"]
        if generic_name not in cache.species_cache.keys():
            species_url = f"https://pokeapi.co/api/v2/pokemon-species/{generic_name}/"
            species_response = requests.get(species_url)

            if species_response.status_code != 200:
                print(species_response.status_code)
                print(species_response.content)
                raise RuntimeError(f"Error when trying to pull species data for: {generic_name}")

            species_entry = species_response.json()

            cache.species_cache[generic_name] = species_entry

        possible_abilities = [
            a["ability"]["name"]
            for a in cache.dex_cache[poke.strip().lower()]["abilities"]
            if (not a["is_hidden"]) and(a["ability"]["name"] not in ["competitive", "slush-rush"])
        ]
        ability = rng.choice(possible_abilities)

        moveset =  []
        for move in cache.dex_cache[poke.strip().lower()]["moves"]:
            for v_details in move["version_group_details"]:
                if v_details["version_group"]["name"] == "black-white":
                    moveset.append((v_details["level_learned_at"], move["move"]["name"]))
        moveset = [m[1] for m in moveset if m[0] <= level]
        if len(moveset) < 4:
            raise RuntimeError(f"Error in loading moveset for: {poke}")
        moveset = list(set(moveset))
        moves = rng.choice(moveset, replace=False, size=4)

        odds_female = cache.species_cache[generic_name]["gender_rate"] / 8
        gender = "Male" if np.random.rand() >= odds_female else "Female"

        evs = [1,1,1,1,1,1]

        ivs = rng.integers(1, 32, size=6)

        pokemon = TeambuilderPokemon(
            species=poke.strip().lower(),
            ability=ability,
            moves=moves,
            gender=gender,
            level=level,
            evs=evs,
            ivs=ivs
        )

        team.append(pokemon)

    showdown_team = Teambuilder.join_team(team)

    return showdown_team

In [4]:
possible_pokemon = [
    "Victini", "Snivy", "Servine", "Serperior", "Tepig", "Pignite", "Emboar", "Oshawott", "Dewott", "Samurott", "Patrat", "Watchog", "Lillipup", "Herdier", "Stoutland", "Purrloin", "Liepard", "Pansage", "Simisage", "Pansear", "Simisear", "Panpour", "Simipour", "Munna", "Musharna", "Pidove", "Tranquill", "Unfezant", "Blitzle", "Zebstrika", "Roggenrola", "Boldore", "Gigalith", "Woobat", "Swoobat", "Drilbur", "Excadrill", "Audino", "Timburr", "Gurdurr", "Conkeldurr", "Tympole", "Palpitoad", "Seismitoad", "Throh", "Sawk", "Sewaddle", "Swadloon", "Leavanny", "Venipede", "Whirlipede", "Scolipede", "Cottonee", "Whimsicott", "Petilil", "Lilligant", "Sandile", "Krokorok", "Krookodile", "Darumaka", "Maractus", "Dwebble", "Crustle", "Scraggy", "Scrafty", "Sigilyph", "Yamask", "Cofagrigus", "Tirtouga", "Carracosta", "Archen", "Archeops", "Trubbish", "Garbodor", "Zorua", "Zoroark", "Minccino", "Cinccino", "Gothita", "Gothorita", "Gothitelle", "Solosis", "Duosion", "Reuniclus", "Ducklett", "Swanna", "Vanillite", "Vanillish", "Vanilluxe", "Deerling", "Sawsbuck", "Emolga", "Karrablast", "Escavalier", "Foongus", "Amoonguss", "Frillish", "Jellicent", "Alomomola", "Joltik", "Galvantula", "Ferroseed", "Ferrothorn", "Klink", "Klang", "Klinklang", "Tynamo", "Eelektrik", "Eelektross", "Elgyem", "Beheeyem", "Litwick", "Lampent", "Chandelure", "Axew", "Fraxure", "Haxorus", "Cubchoo", "Beartic", "Cryogonal", "Shelmet", "Accelgor", "Stunfisk", "Mienfoo", "Mienshao", "Druddigon", "Golett", "Golurk", "Pawniard", "Bisharp", "Bouffalant", "Rufflet", "Braviary", "Vullaby", "Mandibuzz", "Heatmor", "Durant", "Deino", "Zweilous", "Hydreigon", "Larvesta", "Volcarona", "Cobalion", "Terrakion", "Virizion", "Reshiram", "Zekrom", "Kyurem", "Genesect", "Thundurus-Incarnate", "Landorus-Incarnate", "Tornadus-Incarnate", "Darmanitan-Standard", "Keldeo-Ordinary", "Meloetta-Aria", "Basculin-Red-Striped", "Basculin-Blue-Striped"
]

cache = PokeCache(fpath="../data/pokecache.json")

build_random_team(possible_pokemon, random_seed=49036, cache=cache)

'|scolipede||poisonpoint|cut,xscissor,return,swordsdance||1,1,1,1,1,1|Female|17,10,24,5,3,27||80|]|heatmor||gluttony|gigaimpact,bugbite,pursuit,facade||1,1,1,1,1,1|Male|12,,19,26,24,9||80|]|lillipup||vitalspirit|swagger,workup,lick,firefang||1,1,1,1,1,1|Male|23,13,13,,27,4||80|]|hydreigon||levitate|taunt,sunnyday,focusenergy,flashcannon||1,1,1,1,1,1|Male|20,23,30,10,10,5||80|]|simisear||gluttony|rest,grassknot,attract,dig||1,1,1,1,1,1|Male|16,4,30,30,,12||80|]|ferroseed||ironbarbs|poisonjab,leechseed,payback,mirrorshot||1,1,1,1,1,1|Female|16,13,3,3,28,7||80|'

In [8]:
elo_data = pd.read_excel("../data/gen5hackmon_adjustedelo.xlsx")
elo_data

,Elo,Username,W,L,T,Last update,AdjustedELO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1529.710693,Team_0049036,50.0,0.0,0.0,Tue Jan 25 2022 12:26:43 GMT+0900 (Japan Stand...,3000.586360
2,1507.437905,Team_0095251,50.0,0.0,0.0,Tue Jan 25 2022 13:54:14 GMT+0900 (Japan Stand...,2956.897430
3,1497.040762,Team_0036157,52.0,0.0,0.0,Mon Jan 24 2022 09:30:16 GMT+0900 (Japan Stand...,2938.635569
4,1498.012381,Team_0056881,50.0,0.0,0.0,Mon Jan 24 2022 22:48:35 GMT+0900 (Japan Stand...,2938.408900
...,...,...,...,...,...,...,...
86345,1000.000000,Team_0009067,0.0,93.0,0.0,Tue Jan 25 2022 18:56:49 GMT+0900 (Japan Stand...,21.052632
86346,1000.000000,Team_0037050,0.0,97.0,0.0,Tue Jan 25 2022 18:43:17 GMT+0900 (Japan Stand...,20.202020
86347,1000.000000,Team_0023249,0.0,100.0,0.0,Tue Jan 25 2022 17:16:36 GMT+0900 (Japan Stand...,19.607843
86348,1000.000000,Team_0070490,0.0,103.0,0.0,Tue Jan 25 2022 19:34:24 GMT+0900 (Japan Stand...,19.047619


In [10]:
elo_data = elo_data.dropna()
elo_data

,Elo,Username,W,L,T,Last update,AdjustedELO
1,1529.710693,Team_0049036,50.0,0.0,0.0,Tue Jan 25 2022 12:26:43 GMT+0900 (Japan Stand...,3000.586360
2,1507.437905,Team_0095251,50.0,0.0,0.0,Tue Jan 25 2022 13:54:14 GMT+0900 (Japan Stand...,2956.897430
3,1497.040762,Team_0036157,52.0,0.0,0.0,Mon Jan 24 2022 09:30:16 GMT+0900 (Japan Stand...,2938.635569
4,1498.012381,Team_0056881,50.0,0.0,0.0,Mon Jan 24 2022 22:48:35 GMT+0900 (Japan Stand...,2938.408900
5,1489.135680,Team_0006920,55.0,0.0,0.0,Tue Jan 25 2022 06:46:32 GMT+0900 (Japan Stand...,2926.020985
...,...,...,...,...,...,...,...
86345,1000.000000,Team_0009067,0.0,93.0,0.0,Tue Jan 25 2022 18:56:49 GMT+0900 (Japan Stand...,21.052632
86346,1000.000000,Team_0037050,0.0,97.0,0.0,Tue Jan 25 2022 18:43:17 GMT+0900 (Japan Stand...,20.202020
86347,1000.000000,Team_0023249,0.0,100.0,0.0,Tue Jan 25 2022 17:16:36 GMT+0900 (Japan Stand...,19.607843
86348,1000.000000,Team_0070490,0.0,103.0,0.0,Tue Jan 25 2022 19:34:24 GMT+0900 (Japan Stand...,19.047619


In [12]:
elo_data["Team"] = [
    build_random_team(possible_pokemon, random_seed=int(team_id.split("_")[1]), cache=cache)
    for team_id in elo_data["Username"]
]
elo_data

,Elo,Username,W,L,T,Last update,AdjustedELO,Team
1,1529.710693,Team_0049036,50.0,0.0,0.0,Tue Jan 25 2022 12:26:43 GMT+0900 (Japan Stand...,3000.586360,"|scolipede||poisonpoint|cut,xscissor,return,sw..."
2,1507.437905,Team_0095251,50.0,0.0,0.0,Tue Jan 25 2022 13:54:14 GMT+0900 (Japan Stand...,2956.897430,"|garbodor||stench|round,protect,swallow,sunnyd..."
3,1497.040762,Team_0036157,52.0,0.0,0.0,Mon Jan 24 2022 09:30:16 GMT+0900 (Japan Stand...,2938.635569,"|zweilous||hustle|psychup,hypervoice,facade,dr..."
4,1498.012381,Team_0056881,50.0,0.0,0.0,Mon Jan 24 2022 22:48:35 GMT+0900 (Japan Stand...,2938.408900,"|ducklett||keeneye|rest,frustration,watersport..."
5,1489.135680,Team_0006920,55.0,0.0,0.0,Tue Jan 25 2022 06:46:32 GMT+0900 (Japan Stand...,2926.020985,"|kyurem||pressure|fling,rocktomb,dracometeor,h..."
...,...,...,...,...,...,...,...,...
86345,1000.000000,Team_0009067,0.0,93.0,0.0,Tue Jan 25 2022 18:56:49 GMT+0900 (Japan Stand...,21.052632,"|snivy||overgrow|pursuit,calmmind,meanlook,lea..."
86346,1000.000000,Team_0037050,0.0,97.0,0.0,Tue Jan 25 2022 18:43:17 GMT+0900 (Japan Stand...,20.202020,"|woobat||klutz|chargebeam,knockoff,substitute,..."
86347,1000.000000,Team_0023249,0.0,100.0,0.0,Tue Jan 25 2022 17:16:36 GMT+0900 (Japan Stand...,19.607843,"|elgyem||telepathy|psybeam,safeguard,headbutt,..."
86348,1000.000000,Team_0070490,0.0,103.0,0.0,Tue Jan 25 2022 19:34:24 GMT+0900 (Japan Stand...,19.047619,"|yamask||mummy|payback,hiddenpower,return,omin..."


In [13]:
def is_pokemon_included(pokemon, team) -> int:
    if pokemon.strip().lower() in team:
        return 1
    else:
        return 0

In [14]:
for pokemon in possible_pokemon:
    elo_data[f"Contains_{pokemon}"] = [is_pokemon_included(pokemon, team) for team in elo_data["Team"]]

elo_data

C:\Users\Trevor\AppData\Local\Temp/ipykernel_33652/574687766.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  elo_data[f"Contains_{pokemon}"] = [is_pokemon_included(pokemon, team) for team in elo_data["Team"]]


,Elo,Username,W,L,T,Last update,AdjustedELO,Team,Contains_Victini,Contains_Snivy,...,Contains_Kyurem,Contains_Genesect,Contains_Thundurus-Incarnate,Contains_Landorus-Incarnate,Contains_Tornadus-Incarnate,Contains_Darmanitan-Standard,Contains_Keldeo-Ordinary,Contains_Meloetta-Aria,Contains_Basculin-Red-Striped,Contains_Basculin-Blue-Striped
1,1529.710693,Team_0049036,50.0,0.0,0.0,Tue Jan 25 2022 12:26:43 GMT+0900 (Japan Stand...,3000.586360,"|scolipede||poisonpoint|cut,xscissor,return,sw...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,1507.437905,Team_0095251,50.0,0.0,0.0,Tue Jan 25 2022 13:54:14 GMT+0900 (Japan Stand...,2956.897430,"|garbodor||stench|round,protect,swallow,sunnyd...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,1497.040762,Team_0036157,52.0,0.0,0.0,Mon Jan 24 2022 09:30:16 GMT+0900 (Japan Stand...,2938.635569,"|zweilous||hustle|psychup,hypervoice,facade,dr...",0,0,...,0,0,0,0,0,0,0,0,0,0
4,1498.012381,Team_0056881,50.0,0.0,0.0,Mon Jan 24 2022 22:48:35 GMT+0900 (Japan Stand...,2938.408900,"|ducklett||keeneye|rest,frustration,watersport...",0,0,...,0,0,0,0,0,0,0,0,0,0
5,1489.135680,Team_0006920,55.0,0.0,0.0,Tue Jan 25 2022 06:46:32 GMT+0900 (Japan Stand...,2926.020985,"|kyurem||pressure|fling,rocktomb,dracometeor,h...",0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86345,1000.000000,Team_0009067,0.0,93.0,0.0,Tue Jan 25 2022 18:56:49 GMT+0900 (Japan Stand...,21.052632,"|snivy||overgrow|pursuit,calmmind,meanlook,lea...",0,1,...,0,0,0,0,0,0,0,0,0,0
86346,1000.000000,Team_0037050,0.0,97.0,0.0,Tue Jan 25 2022 18:43:17 GMT+0900 (Japan Stand...,20.202020,"|woobat||klutz|chargebeam,knockoff,substitute,...",0,0,...,0,0,0,0,0,0,0,0,0,0
86347,1000.000000,Team_0023249,0.0,100.0,0.0,Tue Jan 25 2022 17:16:36 GMT+0900 (Japan Stand...,19.607843,"|elgyem||telepathy|psybeam,safeguard,headbutt,...",0,0,...,0,0,0,0,0,0,0,0,0,0
86348,1000.000000,Team_0070490,0.0,103.0,0.0,Tue Jan 25 2022 19:34:24 GMT+0900 (Japan Stand...,19.047619,"|yamask||mummy|payback,hiddenpower,return,omin...",0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
elo_results = []
adj_elo_results = []
for pokemon in tqdm(possible_pokemon):
    avg_included_elo = elo_data[elo_data[f"Contains_{pokemon}"] == 1]["Elo"].mean()
    avg_excluded_elo = elo_data[elo_data[f"Contains_{pokemon}"] == 0]["Elo"].mean()
    
    avg_included_adj_elo = elo_data[elo_data[f"Contains_{pokemon}"] == 1]["AdjustedELO"].mean()
    avg_excluded_adj_elo = elo_data[elo_data[f"Contains_{pokemon}"] == 0]["AdjustedELO"].mean()
    
    elo_results.append((pokemon, avg_included_elo, avg_excluded_elo))
    adj_elo_results.append((pokemon, avg_included_adj_elo, avg_excluded_adj_elo))

simple_score = pd.DataFrame(columns=["Pokemon", "IncludeScore", "ExcludeScore"], data=elo_results)
simple_score = simple_score.dropna()
simple_score["Improvement"] = simple_score["IncludeScore"] - simple_score["ExcludeScore"]
simple_score.sort_values(by=["Improvement"], ascending=False)

100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:22<00:00,  7.08it/s]


,Pokemon,IncludeScore,ExcludeScore,Improvement
145,Reshiram,1214.837989,1157.097875,57.740114
146,Zekrom,1210.630163,1157.312429,53.317734
148,Genesect,1205.023921,1157.447583,47.576338
147,Kyurem,1204.891778,1157.481613,47.410165
139,Hydreigon,1202.853773,1157.511234,45.342539
...,...,...,...,...
17,Pansage,1154.243538,1158.998619,-4.755080
10,Patrat,1153.939665,1159.002785,-5.063120
66,Yamask,1153.646375,1159.009812,-5.363436
1,Snivy,1152.863473,1159.030743,-6.167271


In [16]:
adjusted_score = pd.DataFrame(columns=["Pokemon", "IncludeScore", "ExcludeScore"], data=adj_elo_results)
adjusted_score = adjusted_score.dropna()
adjusted_score["Improvement"] = adjusted_score["IncludeScore"] - adjusted_score["ExcludeScore"]
adjusted_score.sort_values(by=["Improvement"], ascending=False)

,Pokemon,IncludeScore,ExcludeScore,Improvement
145,Reshiram,1966.612812,1470.257126,496.355686
146,Zekrom,1946.651260,1471.617194,475.034066
147,Kyurem,1905.850661,1472.815531,433.035130
139,Hydreigon,1889.396242,1473.019900,416.376342
148,Genesect,1884.232307,1473.202648,411.029659
...,...,...,...,...
15,Purrloin,1537.537474,1483.849216,53.688258
10,Patrat,1536.658468,1483.901750,52.756718
17,Pansage,1534.971733,1483.900462,51.071271
1,Snivy,1526.984478,1484.208092,42.776386


In [17]:
elo_data = elo_data.sample(frac=1.0)

In [18]:
X = elo_data[[col for col in elo_data.columns if "Contains_" in col]]
X

,Contains_Victini,Contains_Snivy,Contains_Servine,Contains_Serperior,Contains_Tepig,Contains_Pignite,Contains_Emboar,Contains_Oshawott,Contains_Dewott,Contains_Samurott,...,Contains_Kyurem,Contains_Genesect,Contains_Thundurus-Incarnate,Contains_Landorus-Incarnate,Contains_Tornadus-Incarnate,Contains_Darmanitan-Standard,Contains_Keldeo-Ordinary,Contains_Meloetta-Aria,Contains_Basculin-Red-Striped,Contains_Basculin-Blue-Striped
35653,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52813,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36783,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41602,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81934,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80894,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5814,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23015,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
Y = (elo_data["AdjustedELO"] - elo_data["AdjustedELO"].mean()) / elo_data["AdjustedELO"].std()
Y

35653    0.401796
52813   -0.132172
36783    0.331831
41602    0.222437
81934   -1.730021
           ...   
80894   -1.663717
8561     1.209095
5814     1.280742
23015    0.794053
75984   -1.414677
Name: AdjustedELO, Length: 86349, dtype: float64

In [21]:
X_train = X.iloc[:70000, :]
X_val = X.iloc[70000:, :]

Y_train = Y.iloc[:70000]
Y_val = Y.iloc[70000:]

In [22]:
Y_train

35653    0.401796
52813   -0.132172
36783    0.331831
41602    0.222437
81934   -1.730021
           ...   
44175    0.193851
76460   -1.414677
53361   -0.132172
6021     1.274506
84679   -1.927406
Name: AdjustedELO, Length: 70000, dtype: float64

In [23]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

model = Sequential()

model.add(Dense(64, input_shape=(157,)))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(32))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(1))

model.compile(loss="mse", optimizer="adam")
model.summary()

1 Physical GPUs, 1 Logical GPUs
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                10112     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 batch_normalization_1 (Batc  (None, 32)               128       
 hNormalization)        

In [24]:
model.fit(
    X_train,
    Y_train,
    epochs=10,
    validation_data=(X_val, Y_val)
)

Epoch 1/10
2188/2188 [==============================] - 52s 23ms/step - loss: 0.7395 - val_loss: 0.5312
Epoch 2/10
2188/2188 [==============================] - 58s 26ms/step - loss: 0.5736 - val_loss: 0.5349
Epoch 3/10
2188/2188 [==============================] - 59s 27ms/step - loss: 0.5644 - val_loss: 0.5354
Epoch 4/10
2188/2188 [==============================] - 60s 27ms/step - loss: 0.5621 - val_loss: 0.5327
Epoch 5/10
2188/2188 [==============================] - 57s 26ms/step - loss: 0.5621 - val_loss: 0.5341
Epoch 6/10
2188/2188 [==============================] - 51s 23ms/step - loss: 0.5615 - val_loss: 0.5302
Epoch 7/10
 185/2188 [=>............................] - ETA: 39s - loss: 0.5473

KeyboardInterrupt: 

In [30]:
def fitness_function(team, possible_pokemon, model) -> float:
    team_row = [1 if (p in team) else 0 for p in possible_pokemon]
    
    score = model.predict(np.array(team_row).reshape(1, -1)).squeeze()*elo_data["AdjustedELO"].std() + elo_data["AdjustedELO"].mean()
    
    return score

In [31]:
def find_best_team(pop_size, possible_pokemon, model):
    team_data = []
    print("Building Initial Population...")
    for _ in tqdm(range(pop_size)):
        team = np.random.choice(possible_pokemon, size=6, replace=True)
        fitness_score = fitness_function(team, possible_pokemon, model)

        team_data.append((team, fitness_score))

    team_info = pd.DataFrame(columns=["Team", "Score"], data=team_data)
    team_info = team_info.sort_values(by=["Score"], ascending=False)

    print("Finding Initial seed for Generation 1...")
    seed_population = team_info.head(pop_size // 6)
    best_team = seed_population.iloc[0, 0]
    best_score = seed_population["Score"].max()
    
    return mutate_team(seed_population, best_team, best_score, pop_size, 1, possible_pokemon, model)

In [32]:
def mutate_team(seed_population, best_team, best_score, pop_size, depth, possible_pokemon, model):
    team_data = []
    print(f"Mutating Generation {depth}...")
    for team in tqdm(seed_population["Team"]):
        new_pokemon = np.random.choice([p for p in possible_pokemon if p not in team])

        for i in range(6):
            temp_team = deepcopy(team)
            temp_team[i] = new_pokemon

            fitness_score = fitness_function(temp_team, possible_pokemon, model)
            team_data.append((temp_team, fitness_score))

    team_info = pd.DataFrame(columns=["Team", "Score"], data=team_data)
    team_info = team_info.sort_values(by=["Score"], ascending=False)

    best_new_team = team_info.iloc[0, 0]
    best_new_score = team_info["Score"].max()

    if best_new_score < best_score:
        return best_team
    elif depth > 20:
        return best_new_team
    else:
        best_team = best_new_team
        best_score = best_new_score
        seed_population = team_info.head(pop_size // 6)
        
        return mutate_team(seed_population, best_team, best_score, pop_size, depth+1, possible_pokemon, model)

In [29]:
find_best_team(240)

Building Initial Population...


100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [00:13<00:00, 17.85it/s]


Finding Initial seed for Generation 1...
Mutating Generation 1...


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.30it/s]


Mutating Generation 2...


...

Mutating Generation 6...


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:13<00:00,  3.02it/s]


Mutating Generation 7...


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [00:12<00:00,  3.27it/s]


array(['Zekrom', 'Reshiram', 'Cobalion', 'Virizion', 'Hydreigon',
       'Haxorus'], dtype='<U21')

In [ ]:
fitness_function([
    'Kyurem', 'Druddigon', 'Conkeldurr', 'Reshiram', 'Krookodile','Haxorus'
])

In [33]:
model.save("../models/elo_predictor")

INFO:tensorflow:Assets written to: ../models/elo_predictor\assets


In [34]:
from tensorflow.keras.models import load_model

In [35]:
load_model("../models/elo_predictor").summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                10112     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 batch_normalization_1 (Batc  (None, 32)               128       
 hNormalization)                                        